In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pp
import seaborn as sns

%matplotlib inline

In [2]:
from stats_can import StatsCan

In [3]:
df = pd.read_csv('../data_cleaned.csv')
df['wk_strt_dt'] = pd.to_datetime(df['wk_strt_dt'])

In [ ]:
df['ref_yr_mth'] = [item for item in zip(df['wk_strt_dt'].dt.year, df['wk_strt_dt'].dt.month)]

In [ ]:
econ_df = pd.read_csv('econ_indicators.csv')

In [ ]:
econ_df['date'] = pd.to_datetime(econ_df['date']) 

In [ ]:
sc = StatsCan()
unem_df = sc.vectors_to_df_remote('v2062815', periods = 360)
unem_df.columns = ['unemployment_rate']
unem_df = unem_df.reset_index()

gdp_df = sc.vectors_to_df_remote('v65201210', periods = 360)
gdp_df.columns = ['monthly_gdp']
gdp_df = gdp_df.reset_index()

cpi_df = sc.vectors_to_df_remote('v41690973', periods = 360)
cpi_df.columns = ['monthly_cpi']
cpi_df = cpi_df.reset_index()

econ_df = pd.merge(unem_df, gdp_df, on='refPer', how='inner')
econ_df = econ_df.merge(cpi_df, on='refPer')

# econ_df = pd.concat([unem_df, gdp_df, cpi_df], axis = 1)

In [ ]:
econ_df

In [ ]:
econ_df['ref_date'] = [item for item in zip(econ_df['refPer'].dt.year, econ_df['refPer'].dt.month)]

In [ ]:
pd.merge(df, econ_df, how='left', left_on='ref_yr_mth', right_on='ref_date').drop(['refPer','ref_yr_mth','ref_date'], axis = 1)

In [14]:
# New
def make_dataframe(start_date, end_date, user_data_filepath, user_data_date_column, include_econ_indicators=True):
    '''
    When function is called, returns a Pandas dataframe with time interval every sunday 
    including the first one before the `start_date` and the last one before `end_date`.
    start_date, end_date: input in the yyyy-mm-dd format
    include_econ_indicators: True by default. Pulls unemployment, monthly CPI, and monthly GDP data from Statistics Canada using stats_can API
    user_data_filepath: string with path to the user provided adspend and kpi data
    user_data_date_column: name of column with date. The first and last dates must match `start_date` and `end_date` respectively
    '''
    user_df = pd.read_csv(user_data_filepath)
    user_df[user_data_date_column] = pd.to_datetime(user_df[user_data_date_column])
    user_df['ref_yr_mth'] = [item for item in zip(user_df[user_data_date_column].dt.year, user_df[user_data_date_column].dt.month)]

    start_date_ts = pd.Timestamp(start_date)
    end_date_ts = pd.Timestamp(end_date)

    df = pd.DataFrame()
    df['date'] = pd.date_range(start_date_ts, end_date_ts, freq = 'W-SUN')
    df['ref_date'] = [item for item in zip(df['date'].dt.year, df['date'].dt.month)]

    if include_econ_indicators:
        sc = StatsCan()
        unem_df = sc.vectors_to_df_remote('v2062815', periods = 360)
        unem_df.columns = ['unemployment_rate']
        unem_df = unem_df.reset_index()

        gdp_df = sc.vectors_to_df_remote('v65201210', periods = 360)
        gdp_df.columns = ['monthly_gdp']
        gdp_df = gdp_df.reset_index()

        cpi_df = sc.vectors_to_df_remote('v41690973', periods = 360)
        cpi_df.columns = ['monthly_cpi']
        cpi_df = cpi_df.reset_index()

        econ_df = pd.merge(unem_df, gdp_df, on='refPer', how='inner')
        econ_df = econ_df.merge(cpi_df, on='refPer')


        econ_df['refPer_yr_mth'] = [item for item in zip(econ_df['refPer'].dt.year, econ_df['refPer'].dt.month)]

        df_merge_orig_gdp = pd.merge(user_df, 
                                econ_df, 
                                how = 'left',
                                left_on='ref_yr_mth', 
                                right_on='refPer_yr_mth')

        return df_merge_orig_gdp.drop(['refPer_yr_mth','ref_yr_mth','refPer',], axis = 1)
    else:
        return user_df

In [15]:
mk_df = make_dataframe('2014-08-03', '2018-07-29', '../data_cleaned.csv', 'wk_strt_dt')
mk_df

,wk_strt_dt,yr_nbr,qtr_nbr,prd,wk_nbr,wk_in_yr_nbr,mdip_dm,mdip_inst,mdip_nsp,mdip_auddig,...,va_pub_0.15,va_pub_0.2,va_pub_0.25,va_pub_0.3,Electronics,unemployment_rate_x,is_holiday_week,unemployment_rate_y,monthly_gdp,monthly_cpi
0,2014-08-03,2014,3,7,1,27,4863885,29087520,2421933,692315,...,1,0,0,0,63,7.0,0,7.0,1809135.0,125.7
1,2014-08-10,2014,3,7,2,28,20887502,8345120,3984494,475810,...,1,0,0,0,54,7.0,0,7.0,1809135.0,125.7
2,2014-08-17,2014,3,7,3,29,11097724,17276800,1846832,784732,...,1,1,0,0,54,7.0,0,7.0,1809135.0,125.7
3,2014-08-24,2014,3,7,4,30,1023446,18468480,2394834,1032301,...,1,1,0,0,45,7.0,0,7.0,1809135.0,125.7
4,2014-08-31,2014,3,8,1,31,21109811,26659920,3312008,400456,...,0,1,0,0,52,7.0,1,7.0,1809135.0,125.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,2018-07-01,2018,2,5,5,22,13038,13828188,387,2135131,...,0,1,0,0,41,5.8,1,5.8,1952354.0,134.3
205,2018-07-08,2018,2,6,1,23,0,3714294,0,1448480,...,0,1,1,0,45,5.8,1,5.8,1952354.0,134.3
206,2018-07-15,2018,2,6,2,24,2225092,5102353,0,1307764,...,0,0,1,0,47,5.8,0,5.8,1952354.0,134.3
207,2018-07-22,2018,2,6,3,25,17544331,1478566,1891,510539,...,0,1,0,0,44,5.8,0,5.8,1952354.0,134.3


In [ ]:
## Original
# def make_dataframe(start_date, end_date, include_econ_indicators=True, dataset_info = None):
#     '''
#     When function is called, returns a Pandas dataframe with time interval every sunday 
#     including the first one before the `start_date` and the last one before `end_date`.
#     start_date, end_date: input in the yyyy-mm-dd format
#     include_econ_indicators: True by default. Pulls unemployment, monthly CPI, and monthly GDP data from Statistics Canada using stats_can API
#     dataset_info: dict with the following information- `filepath`, `ad_spend_list`, `date_column`
#     '''
#     start_date_ts = pd.Timestamp(start_date)
#     end_date_ts = pd.Timestamp(end_date)

#     df = pd.DataFrame()
#     df['date'] = pd.date_range(start_date_ts, end_date_ts, freq = 'W-SUN')
#     df['date_row_yr_mth'] = [item for item in zip(df['date'].dt.year, df['date'].dt.month)]

#     if include_econ_indicators:
#         sc = StatsCan()
#         unem_df = sc.vectors_to_df_remote('v2062815', periods = 360)
#         unem_df.columns = ['unemployment_rate']
#         unem_df = unem_df.reset_index()

#         gdp_df = sc.vectors_to_df_remote('v65201210', periods = 360)
#         gdp_df.columns = ['monthly_gdp']
#         gdp_df = gdp_df.reset_index()

#         cpi_df = sc.vectors_to_df_remote('v41690973', periods = 360)
#         cpi_df.columns = ['monthly_cpi']
#         cpi_df = cpi_df.reset_index()

#         econ_df = pd.merge(unem_df, gdp_df, on='refPer', how='inner')
#         econ_df = econ_df.merge(cpi_df, on='refPer')

#         econ_df['refPer_yr_mth'] = [item for item in zip(econ_df['refPer'].dt.year, econ_df['refPer'].dt.month)]

#         df_merge_orig_gdp = pd.merge(df, 
#                                 econ_df, 
#                                 how = 'inner', 
#                                 right_on='refPer_yr_mth',
#                                 left_on='date_row_yr_mth')

#         df_merge_orig_gdp = df_merge_orig_gdp.drop(['refPer_yr_mth','date_row_yr_mth','refPer'], axis = 1)

#         return df_merge_orig_gdp

#     else:
#         return df

# #     if dataset_info is not None:
# #         filepath = dataset_info['filepath']
# #         date_column = dataset_info['date_column']
# #         ad_spend_list = dataset_info['ad_spend_list']

# #         data = pd.read_csv(filepath)
# #         data[date_column] = pd.to_datetime(data[date_column])
# #         mdsp_cols = [date_column] + ad_spend_list
# #         spending_data = data[mdsp_cols]

# #         output_df = pd.merge(output_df, spending_data, left_on='date', right_on=date_column)
# #     else:
# #         output_df = df.drop('date_row_yr_mth', axis = 1)


#     return output_df

In [ ]:
start_date = '2019-08-01'
end_date = '2020-08-01'

dataset_info = {
    'filepath': '../data_cleaned.csv',
    'date_column': 'wk_strt_dt',
    'ad_spend_list': ['mdsp_dm','mdsp_inst','mdsp_nsp','mdsp_auddig','mdsp_audtr','mdsp_vidtr','mdsp_viddig','mdsp_so','mdsp_on','mdsp_sem']
}

In [ ]:
df = make_dataframe(start_date, end_date, include_econ_indicators=True)

In [ ]:
df

In [ ]:
def hill_function(k,s,x):
    return 1 / (1 + (x / k)**-s)

In [ ]:
x = np.linspace(0,1,num=100)
y = hill_function(2, 1.5, x)

In [ ]:
pp.plot(x,y)

In [ ]:
%ls ..

In [ ]:
df = pd.read_csv('../data_cleaned.csv', infer_datetime_format=True)
df

In [ ]:
df.columns

In [ ]:
df['wk_strt_dt'] = pd.to_datetime(df['wk_strt_dt'])

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X = df.loc[:,['sales','mdsp_viddig']]
X_sc = sc.fit_transform(X)

In [ ]:
X_sc

In [ ]:
df.loc[:,'mroas']

In [ ]:
pp.scatter(X_sc[1], X_sc[0])

In [ ]:
X_sc[0][0]

In [ ]:
xs = df['mdsp_viddig'].sort_values()
ys = hill_function(8, 2, xs)

pp.plot(xs,ys)

In [ ]:
pp.scatter(xs, ys)

In [ ]:
xs

In [ ]:
xs.shape

In [ ]:
ys.shape